In [8]:
#Amkes Code umgeschrieben, da ich gerade nur die Inzidenz brauche und sie noch weitere Werte herausgezogen hat
import requests, json

def get_data_by_id(id):
    url = "https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/rki_key_data_v/FeatureServer/0/query?"
    
    parameter = {
        'referer':'https://www.mywebapp.com',
        'user-agent':'python-requests/2.9.1',
        'where': f'AdmUnitId = {id}', # Welche landkreise sollen zurück gegeben werden
        'outFields': '*', # Rückgabe aller Felder
        'returnGeometry': False, # Keine Geometrien
        'f':'json', # Rückgabeformat, hier JSON
        'cacheHint': True # Zugriff über CDN anfragen
    }
    result3 = requests.get(url=url, params=parameter) #Anfrage absetzen
    resultjson3 = json.loads(result3.text) # Das Ergebnis JSON als Python Dictionary laden
    return float(resultjson3['features'][0]['attributes']['Inz7T'])

In [2]:
import pandas as pd

# als dictionary
große_staedte= {'AdminUnitId': {
  'Berlin': 11,
  'Berlin-Mitte': 11001,  
  'München': 9162,
  'Hamburg': 2000,
  'Köln': 5315,
  'Frankfurt': 6412,
  'Stuttgart': 8111,
  'Düsseldurf': 5111,
  'Dortmund': 5913,
  'Essen': 5113, }}

# umwandlung in dataframe
df_gr_staedte= pd.DataFrame(große_staedte)
df_gr_staedte

,AdminUnitId
Berlin,11
Berlin-Mitte,11001
Dortmund,5913
Düsseldurf,5111
Essen,5113
Frankfurt,6412
Hamburg,2000
Köln,5315
München,9162
Stuttgart,8111


In [3]:
berlin_data= get_data_by_id(11)
berlin_mitte_data= get_data_by_id(11001)
dortmund_data= get_data_by_id(5913)
düsseldorf_data= get_data_by_id(5111)
essen_data= get_data_by_id(5113)
frankfurt_data= get_data_by_id(6412)
hamburg_data= get_data_by_id(2000)
köln_data= get_data_by_id(5315)
münchen_data= get_data_by_id(9162)
stuttgart_data= get_data_by_id(6412)
#print(berlin_data)
berlin_mitte_data_string = json.dumps(berlin_mitte_data)
print(berlin_mitte_data_string)
berlin_mitte_data_float = float(berlin_mitte_data_string);
print(berlin_mitte_data_float)

373.0
373.0


In [9]:
#Verbindung zur Datenbank aufbauen
# Importing module
import mysql.connector
 
# Creating connection object
inzdb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "",
    db = "inzidenzen"
)
 
# Printing the connection object
print(inzdb)

In [7]:
#Einfügen des tagesaktuellen Inzidenzwerte der zehn größten Städte
#Tabelle Berlin-Mitte ist zum Testen
from datetime import datetime
from datetime import timedelta
from time import mktime
now = datetime.now()
current_date = now.strftime('%Y-%m-%d') #Ausgabe formatieren
cursor = inzdb.cursor()

#cursor.execute("INSERT INTO `inzidenzen_deutschland_berlin_mitte`(`datum`, `inzidenz`) VALUES (NOW(), 355.5)")

add_inzidenz_berlin_mitte="""INSERT INTO inzidenzen_deutschland_berlin_mitte (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_berlin_mitte, (current_date, berlin_mitte_data))

#add_inzidenz_berlin_mitte="""INSERT INTO inzidenzen_deutschland_berlin_mitte (datum, inzidenz) VALUES(%s, %s)"""
#cursor.execute(add_inzidenz_berlin_mitte, ('2021-11-18', '1.1'))

add_inzidenz_berlin="""INSERT INTO inzidenzen_deutschland_berlin (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_berlin, (current_date, berlin_data))

add_inzidenz_hamburg="""INSERT INTO inzidenzen_deutschland_hamburg (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_hamburg, (current_date, hamburg_data))

add_inzidenz_köln="""INSERT INTO inzidenzen_deutschland_koeln (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_köln, (current_date, köln_data))

add_inzidenz_münchen="""INSERT INTO inzidenzen_deutschland_muenchen (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_münchen, (current_date, münchen_data))

add_inzidenz_frankfurt="""INSERT INTO inzidenzen_deutschland_frankfurt (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_frankfurt, (current_date, frankfurt_data))

add_inzidenz_düsseldorf="""INSERT INTO inzidenzen_deutschland_duesseldorf (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_düsseldorf, (current_date, düsseldorf_data))

add_inzidenz_dortmund="""INSERT INTO inzidenzen_deutschland_dortmund (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_dortmund, (current_date, dortmund_data))

add_inzidenz_stuttgart="""INSERT INTO inzidenzen_deutschland_stuttgart (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_stuttgart, (current_date, stuttgart_data))

add_inzidenz_essen="""INSERT INTO inzidenzen_deutschland_essen (datum, inzidenz) VALUES(%s, %s)"""
cursor.execute(add_inzidenz_essen, (current_date, essen_data))

cursor.close();
inzdb.commit()

#Datensätze auslesen
#cursor = inzdb.cursor()
#cursor.execute("select * from inzidenzen_deutschland_hamburg")
#cursor.execute("select * from inzidenzen_deutschland_koeln")
#cursor.execute("select * from inzidenzen_deutschland_muenchen")
#result = cursor.fetchall()
#cursor.close()

#for data in result:
#    print("Datum: " + str(data[0]) + "; Inzidenz: " + str(data[1]))

In [10]:
#Löschen von allen Werten, die mehr als 14 Tage zurückliegen

import datetime as d
from datetime import datetime
from datetime import timedelta
from time import mktime
now = datetime.now()
current_date = now.strftime('%Y-%m-%d') #Ausgabe formatieren
cursor = inzdb.cursor()

#print (d.date.today())
delete_date=d.date.today() - d.timedelta(14)
print("Es wurden alle Werte gelöscht, die vor diesem Datum liegen:")
print (delete_date)


delete_inzidenz_berlin_mitte="""DELETE FROM inzidenzen_deutschland_berlin_mitte WHERE datum<%s"""
cursor.execute(delete_inzidenz_berlin_mitte, (delete_date, ))

delete_inzidenz_berlin="""DELETE FROM inzidenzen_deutschland_berlin WHERE datum<%s"""
cursor.execute(delete_inzidenz_berlin, (delete_date, ))

delete_inzidenz_dortmund="""DELETE FROM inzidenzen_deutschland_dortmund WHERE datum<%s"""
cursor.execute(delete_inzidenz_dortmund, (delete_date, ))

delete_inzidenz_duesseldorf="""DELETE FROM inzidenzen_deutschland_duesseldorf WHERE datum<%s"""
cursor.execute(delete_inzidenz_duesseldorf, (delete_date, ))

delete_inzidenz_essen="""DELETE FROM inzidenzen_deutschland_essen WHERE datum<%s"""
cursor.execute(delete_inzidenz_essen, (delete_date, ))

delete_inzidenz_frankfurt="""DELETE FROM inzidenzen_deutschland_frankfurt WHERE datum<%s"""
cursor.execute(delete_inzidenz_frankfurt, (delete_date, ))

delete_inzidenz_hamburg="""DELETE FROM inzidenzen_deutschland_hamburg WHERE datum<%s"""
cursor.execute(delete_inzidenz_hamburg, (delete_date, ))

delete_inzidenz_koeln="""DELETE FROM inzidenzen_deutschland_koeln WHERE datum<%s"""
cursor.execute(delete_inzidenz_koeln, (delete_date, ))

delete_inzidenz_muenchen="""DELETE FROM inzidenzen_deutschland_muenchen WHERE datum<%s"""
cursor.execute(delete_inzidenz_muenchen, (delete_date, ))

delete_inzidenz_stuttgart="""DELETE FROM inzidenzen_deutschland_stuttgart WHERE datum<%s"""
cursor.execute(delete_inzidenz_stuttgart, (delete_date, ))

cursor.close()
inzdb.commit()

#Werte auslesen
#cursor = inzdb.cursor()
#cursor.execute("select * from inzidenzen_deutschland_berlin WHERE datum=2021-12-09")
#result = cursor.fetchall()
#cursor.close()

#for data in result:
#    print("Datum: " + str(data[0]) + "; Inzidenz: " + str(data[1]))
    


Löschen aller werte, die vor diesem Datum liegen:
2021-11-26
